In [1]:
#IMPORT STUFF
#Import libraries:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

/home/isabelle_lim98_gmail_com/.pyenv/versions/3.6.6/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/isabelle_lim98_gmail_com/.pyenv/versions/3.6.6/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
#Read the data
apptrain = pd.read_csv('application_train.csv')
apptest = pd.read_csv('application_test.csv')
bur = pd.read_csv('bureau.csv')
ccb = pd.read_csv('credit_card_balance.csv')
bb = pd.read_csv('bureau_balance.csv')
ip = pd.read_csv('installments_payments.csv')
pos = pd.read_csv('POS_CASH_balance.csv')
pa = pd.read_csv('previous_application.csv')

In [3]:
#One Hot Encoding all objects

categorical_columns = [col for col in apptrain.columns if apptrain[col].dtype == 'object']
apptrain = pd.get_dummies(apptrain, columns= categorical_columns, dummy_na= False)

categorical_columns1 = [col for col in apptest.columns if apptest[col].dtype == 'object']
apptest = pd.get_dummies(apptest, columns= categorical_columns1, dummy_na= False)

categorical_columns2 = [col for col in bur.columns if bur[col].dtype == 'object']
bur = pd.get_dummies(bur, columns= categorical_columns2, dummy_na= False)

categorical_columns3 = [col for col in ccb.columns if ccb[col].dtype == 'object']
ccb = pd.get_dummies(ccb, columns= categorical_columns3, dummy_na= False)

categorical_columns4 = [col for col in bb.columns if bb[col].dtype == 'object']
bb = pd.get_dummies(bb, columns= categorical_columns4, dummy_na= False)

categorical_columns5 = [col for col in ip.columns if ip[col].dtype == 'object']
ip = pd.get_dummies(ip, columns= categorical_columns5, dummy_na= False)

categorical_columns6 = [col for col in pos.columns if pos[col].dtype == 'object']
pos = pd.get_dummies(pos, columns= categorical_columns6, dummy_na= False)

categorical_columns7 = [col for col in pa.columns if pa[col].dtype == 'object']
pa = pd.get_dummies(pa, columns= categorical_columns7, dummy_na= False)

In [4]:
#Merge bur and bb
newbur = pd.merge(left=bur,right=bb, how='left', on='SK_ID_BUREAU')
print(newbur.head())

   SK_ID_CURR  SK_ID_BUREAU  DAYS_CREDIT  CREDIT_DAY_OVERDUE  \
0      215354       5714462         -497                   0   
1      215354       5714463         -208                   0   
2      215354       5714464         -203                   0   
3      215354       5714465         -203                   0   
4      215354       5714466         -629                   0   

   DAYS_CREDIT_ENDDATE  DAYS_ENDDATE_FACT  AMT_CREDIT_MAX_OVERDUE  \
0               -153.0             -153.0                     NaN   
1               1075.0                NaN                     NaN   
2                528.0                NaN                     NaN   
3                  NaN                NaN                     NaN   
4               1197.0                NaN                 77674.5   

   CNT_CREDIT_PROLONG  AMT_CREDIT_SUM  AMT_CREDIT_SUM_DEBT    ...     \
0                   0         91323.0                  0.0    ...      
1                   0        225000.0             171342

In [13]:
#Groupby and Reindexing
Label_1=[s+'_'+l for s in newbur.columns.tolist() if s!='SK_ID_CURR' for l in ['mean','count','median','max', 'min']]
avg_bur=newbur.groupby('SK_ID_CURR').agg(['mean','count','median','max', 'min']).reset_index()
avg_bur.columns=['SK_ID_CURR']+Label_1

Label_2=[s+'_'+l for s in ccb.columns.tolist() if s!='SK_ID_CURR' for l in ['mean','count','median','max', 'min']]
avg_ccb=ccb.groupby('SK_ID_CURR').agg(['mean','count','median','max', 'min']).reset_index()
avg_ccb.columns=['SK_ID_CURR']+Label_2

Label_3=[s+'_'+l for s in pos.columns.tolist() if s not  in ['SK_ID_PREV','SK_ID_CURR'] for l in ['mean','count','median','max', 'min']]
avg_pos=pos.groupby(['SK_ID_PREV','SK_ID_CURR'])\
            .agg(['mean','count','median','max','min']).groupby(level='SK_ID_CURR')\
            .agg('mean').reset_index()
avg_pos.columns=['SK_ID_CURR']+Label_3

Label_4=[s+'_'+l for s in pa.columns.tolist() if s!='SK_ID_CURR' for l in ['mean','count','median','max', 'min']]
avg_pa=pa.groupby('SK_ID_CURR').agg(['mean','count','median','max', 'min']).reset_index()
avg_pa.columns=['SK_ID_CURR']+Label_4

Label_5=[s+'_'+l for s in ip.columns.tolist() if s!='SK_ID_CURR' for l in ['mean','count','median','max', 'min']]
avg_ip=ip.groupby('SK_ID_CURR').agg(['mean','count','median','max', 'min']).reset_index()
avg_ip.columns=['SK_ID_CURR']+Label_5

MemoryError: 

In [12]:
y = apptrain.TARGET.copy()

newapptrain=(apptrain.drop(labels=['TARGET'],axis=1).append(apptest)
         .merge(avg_bur,on='SK_ID_CURR',how='left')
         .merge(avg_ccb,on='SK_ID_CURR',how='left')
         .merge(avg_pos,on='SK_ID_CURR',how='left')
         .merge(avg_pa,on='SK_ID_CURR',how='left')
         .merge(avg_ip,on='SK_ID_CURR',how='left'))

/home/isabelle_lim98_gmail_com/.pyenv/versions/3.6.6/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


NameError: name 'avg_bur' is not defined

In [7]:
newapptrain = newapptrain.fillna(newapptrain.median())

In [8]:
for col in newapptrain:
    if ((newapptrain[col].isnull().values.ravel().sum()) == 0):
        pass
    else:
        print(col)
        print((newapptrain[col].isnull().values.ravel().sum()))
print('Above are Missing Values found!')

Above are Missing Values found!


In [8]:
#Separating the dataframe
newapptrain.drop(labels=['SK_ID_CURR'],axis=1,inplace=True)
train=newapptrain.iloc[:apptrain.shape[0],:].copy()
test=newapptrain.iloc[apptrain.shape[0]:,:].copy()

In [9]:
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds, show_progress=False)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['TARGET'],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print("\nModel Report")
    print("Accuracy : %.4g" % metrics.accuracy_score(dtrain['TARGET'].values, dtrain_predictions))
    print("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['TARGET'], dtrain_predprob))
                    
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

In [11]:
target = apptrain.TARGET.copy()
train['TARGET'] = target
train['SK_ID_CURR'] = apptrain.SK_ID_CURR


In [12]:
train.set_index('SK_ID_CURR')

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,...,AMT_INSTALMENT_count,AMT_INSTALMENT_median,AMT_INSTALMENT_max,AMT_INSTALMENT_min,AMT_PAYMENT_mean,AMT_PAYMENT_count,AMT_PAYMENT_median,AMT_PAYMENT_max,AMT_PAYMENT_min,TARGET
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,24700.5,406597.5,351000.0,202500.000,0.0,0.0,0.0,0.0,0.0,1.0,...,8.0,9251.7750,53093.745,9251.775,14732.021250,8.0,9251.7750,53093.745,9251.775,1
100003,35698.5,1293502.5,1129500.0,270000.000,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,35652.4875,560835.360,6662.970,95408.622000,10.0,35652.4875,560835.360,6662.970,0
100004,6750.0,135000.0,135000.0,67500.000,0.0,0.0,0.0,0.0,0.0,0.0,...,8.0,9524.7000,19661.940,5264.505,11274.615000,8.0,9146.3400,19766.745,3917.430,0
100006,29686.5,312682.5,297000.0,135000.000,0.0,0.0,0.0,0.0,0.0,1.0,...,7.0,29027.5200,691786.890,2482.920,114142.133571,7.0,29027.5200,691786.890,2482.920,0
100007,21865.5,513000.0,513000.0,121500.000,0.0,0.0,0.0,0.0,0.0,0.0,...,15.0,16509.6000,22678.785,1834.290,13553.466000,15.0,16509.6000,22678.785,1834.290,0
100008,27517.5,490495.5,454500.0,99000.000,0.0,0.0,0.0,1.0,0.0,1.0,...,10.0,15015.0600,25309.575,8019.090,14893.866000,10.0,15015.0600,25309.575,8019.090,0
100009,41301.0,1560726.0,1395000.0,171000.000,0.0,0.0,1.0,1.0,0.0,2.0,...,13.0,8821.2600,17341.605,6155.280,9526.863462,13.0,8821.2600,17341.605,6155.280,0
100010,42075.0,1530000.0,1530000.0,360000.000,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,27463.4100,27463.410,27463.410,27463.410000,1.0,27463.4100,27463.410,27463.410,0
100011,33826.5,1019610.0,913500.0,112500.000,0.0,0.0,0.0,0.0,0.0,1.0,...,48.0,14588.5500,46485.000,563.355,13552.676250,48.0,9000.0000,46485.000,449.685,0


In [ ]:
clf = XGBClassifier(learning_rate =0.01,
n_estimators=10000,
max_depth=4, 
min_child_weight=4,
subsample=0.8, 
colsample_bytree=0.8,
objective= 'binary:logistic', 
nthread=4,
scale_pos_weight=2, 
seed=27)
clf.fit(train, target)

In [ ]:
predictions = clf.predict_proba(test)[:,1]

#Choosing predictors
predictors = [x for x in train.columns if x not in ['TARGET', 'SK_ID_CURR']]
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb1, train, predictors)

def objective(space):

    clf = xgb.XGBClassifier(n_estimators = 10000,
                            max_depth = space['max_depth'],
                            min_child_weight = space['min_child_weight'],
                            subsample = space['subsample'])

    eval_set  = [( train, y), ( test, y_valid)]

    clf.fit(train[col_train], y,
            eval_set=eval_set, eval_metric="auc",
            early_stopping_rounds=30)

    pred = clf.predict_proba(valid)[:,1]
    auc = roc_auc_score(y_valid, pred)
    print ("SCORE:", auc)

    return{'loss':1-auc, 'status': STATUS_OK }

from sklearn.metrics import roc_auc_score
import xgboost as xgb
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

space ={
        'max_depth': hp.quniform("x_max_depth", 5, 30, 1),
        'min_child_weight': hp.quniform ('x_min_child', 1, 10, 1),
        'subsample': hp.uniform ('x_subsample', 0.8, 1)
    }

trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=100,
            trials=trials)

print (best)

params = {list(objective = "binary:logistic",
                 booster = "gbtree",
                 eval_metric = "auc",
                 nthread = 7,
                 eta = 0.05,
                 max_depth = 10,
                 gamma = 0.9,
                 subsample = 0.8,
                 colsample_bytree = 0.8,
                 scale_pos_weight = 50,
                 nrounds = 100)}

In [32]:
my_model = XGBClassifier()
# Add silent=True to avoid printing out updates with each cycle
my_model.fit(mergedapptrain, y_train, verbose=False)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [33]:
predictions = my_model.predict_proba(mergedapptest)[:,1]

In [35]:
cheating = np.hstack([mergedapptest.SK_ID_CURR, predictions]).reshape((-1, 2), order='F')

submission = pd.DataFrame({
      "SK_ID_CURR": cheating[:,0],
      "TARGET": cheating[:,1]
  })

submission.SK_ID_CURR = submission.SK_ID_CURR.astype(int)
submission.TARGET = submission.TARGET.astype(float)

submission.to_csv("Isabelle Kaggle Submission6.csv", index=False)

In [36]:
print(submission)

       SK_ID_CURR    TARGET
0          100001  0.064857
1          100005  0.113647
2          100013  0.020808
3          100028  0.045105
4          100038  0.128227
5          100042  0.044978
6          100057  0.023502
7          100065  0.050904
8          100066  0.022504
9          100067  0.118888
10         100074  0.063513
11         100090  0.062580
12         100091  0.156045
13         100092  0.087865
14         100106  0.058508
15         100107  0.137867
16         100109  0.064464
17         100117  0.041604
18         100128  0.083514
19         100141  0.035409
20         100150  0.026039
21         100168  0.017501
22         100169  0.051580
23         100170  0.059526
24         100171  0.039637
25         100172  0.104834
26         100184  0.095956
27         100187  0.057750
28         100212  0.068557
29         100222  0.076691
...           ...       ...
48714      455963  0.042645
48715      455965  0.026677
48716      456007  0.309978
48717      456008  0